## Quick Checks on Data

The snippets below doesn't reflect any real scenario. Its intent is only serving as a quick consult. They'll be constantly updated as new ideas come up. 

### 0 - Loading the Data 

In [34]:
# importing main libraries
import numpy as np
import pandas as pd 

In [46]:
#setting variables for data frame
quick_dataset = pd.read_csv('data/sample_dataset.csv')

quick_df = pd.DataFrame(quick_dataset)

print(quick_df)

   TransactionID CustomerName Product  Price  Quantity TransactionDate
0            101    Jane Rust  Laptop   1200       1.0      2024-12-01
1            102   june young   Phone    800       2.0      2024/12/01
2            103    Jane Rust  Laptop   1200       NaN      01-12-2024
3            104          NaN  Tablet   -300       1.0             NaN
4            105   JUNE YOUNG   Phone    850       1.0      2024-12-01


As a good practice, index should start on number 1, instead of 0 and the moment to have it fixed is before starting the cleaning. 

In [48]:
# Adjusting the index to start at 1
quick_df.index += 1

# And then overwriting the original file
quick_df.to_csv("data/sample_dataset.csv", index=True)

In [50]:
# some basic information on the dataframe
quick_df.info()
print(quick_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 1 to 5
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   TransactionID    5 non-null      int64  
 1   CustomerName     4 non-null      object 
 2   Product          5 non-null      object 
 3   Price            5 non-null      int64  
 4   Quantity         4 non-null      float64
 5   TransactionDate  4 non-null      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 372.0+ bytes
   TransactionID CustomerName Product  Price  Quantity TransactionDate
1            101    Jane Rust  Laptop   1200       1.0      2024-12-01
2            102   june young   Phone    800       2.0      2024/12/01
3            103    Jane Rust  Laptop   1200       NaN      01-12-2024
4            104          NaN  Tablet   -300       1.0             NaN
5            105   JUNE YOUNG   Phone    850       1.0      2024-12-01


So it's possible to see some interesting findings in a glance, for instance: 
- this ia a store data,
- there are six columns,
- some cleaning will be necessary, such as:  NaN, negative values where it shouldn't be, duplicate, different date formats, and so on. 
- TransactionData data type is an object and it should be datetime type. 

### 1 - Check for Missing Values

In [52]:
# Checking and summing missing values
missing_values = quick_df.isnull().sum()
print(missing_values)

TransactionID      0
CustomerName       1
Product            0
Price              0
Quantity           1
TransactionDate    1
dtype: int64


### 2 - Identifying Incorrect Data Types 

We've already noticed that TransactionDate column has an incorrect data type. Let's try to figure out whether there are others. 


In [54]:
print("Data Types:\n", quick_df.dtypes)

Data Types:
 TransactionID        int64
CustomerName        object
Product             object
Price                int64
Quantity           float64
TransactionDate     object
dtype: object


### 3 - Consistent Format for Dates

On TransactionDate column the date formats are not consistent. We need to fix it up. 

In [56]:
# as this is a quick check, let's create an extra dataframe, so we can handle this issue properly later on
NaT_qdf = quick_df.copy()
NaT_qdf["TransactionDate"] = pd.to_datetime(NaT_qdf["TransactionDate"], errors="coerce")
print(NaT_qdf["TransactionDate"])

1   2024-12-01
2          NaT
3          NaT
4          NaT
5   2024-12-01
Name: TransactionDate, dtype: datetime64[ns]


*NaT* stands for "Not a Time", so it means that the values on the rows 2, 3 and 4, weren't recognized as any date format. Later on will address this properly, as above mentioned. For the time being, the TransactionDate column won't be touched. 

### 4 - Outliers 

We know that domain knowledge is necessary to search for outliers, but it's pretty straight forward applicable to this case, as there is no negative prices.

In [63]:
outliers = quick_df[quick_df["Price"] < 0]
print("Outliers:\n", outliers)

Outliers:
    TransactionID CustomerName Product  Price  Quantity TransactionDate
3            104          NaN  Tablet   -300       1.0             NaT


### 5 - Finding Duplicates

Certainly, in a first view, if we run an automatic check on all the columns, TransactionDate will show duplicates, which doesn't make sense as some records of sales could have happened in the same day. That said, this column will be excluded from this check. 

In [31]:
duplicates = quick_df.duplicated(subset=["CustomerName", "Product"], keep=False)
print("Duplicate Records\n", quick_df[duplicates])

Duplicate Records
    TransactionID CustomerName Product  Price  Quantity TransactionDate
0            101    Jane Rust  Laptop   1200       1.0      2024-12-01
2            103    Jane Rust  Laptop   1200       NaN      01-12-2024


Jane Rust appears twice in this dataset, possibly so do June Yung, but this customer name is typed in capital letters. Next topic this issue will be addressed.  

### 6 - Text Data Standardization

In standardization, it is possible to remove extra spaces and ensuring proper capitalization, as it will be right now done on CustomerName column. 

In [66]:
quick_df["CustomerName"] = quick_df["CustomerName"].str.strip().str.title()
print(quick_df["CustomerName"])

1     Jane Rust
2    June Young
3     Jane Rust
4           NaN
5    June Young
Name: CustomerName, dtype: object


##### *** Running again the previous code for duplicate verification purposes. 

In [79]:
duplicates = quick_df.duplicated(subset=["CustomerName", "Product"], keep=False)
print("Duplicate Records\n", quick_df[duplicates])

Duplicate Records
    TransactionID CustomerName Product  Price  Quantity TransactionDate
1            101    Jane Rust  Laptop   1200       1.0      2024-12-01
2            102   June Young   Phone    800       2.0      2024/12/01
3            103    Jane Rust  Laptop   1200       NaN      01-12-2024
5            105   June Young   Phone    850       1.0      2024-12-01


Jane Rust on TransactionID 101 and 103 seems to be a duplicate, as they have the same day, product and price. On the other side, June Yung doesn't, as the price aren't the same, neither Quantity values. 

### 7 - Validate Data Ranges

Prices need to lie within an expected range, or at least no negative price. Once the stakeholders are consulted about the highest product price, we can stablish a maximum price, and obviously 0 will be the minimum. A maximum amount for our purpose would be 5000, so anything outside of a range of 0 and 5000 will be flagged. 

In [83]:
invalid_prices = quick_df[~quick_df["Price"].between(0, 5000)]
print("Invalid Prices:\n", invalid_prices)

Invalid Prices:
    TransactionID CustomerName Product  Price  Quantity TransactionDate
4            104          NaN  Tablet   -300       1.0             NaN


So, on row with TransactionID 104, there is a negative price. Probably a typo. 

### 8 - Unique Values per Column

How many times each product appears? 

In [90]:
# using "value-counts()" will provide use the answer, spotting the typos and/or anomalies in categorical data
unique_products =quick_df["Product"].value_counts()
print("Unique Products:/n", unique_products)

Unique Products:/n Product
Laptop    2
Phone     2
Tablet    1
Name: count, dtype: int64


### 9 - Insconsistent Formatting Across Columns 

In [99]:
inconsistent_words = quick_df["CustomerName"].str.contains(r"A-Z]{2,}", na=False)
print("Inconsistent Formatting in Names:\n", quick_df[inconsistent_words])

Inconsistent Formatting in Names:
 Empty DataFrame
Columns: [TransactionID, CustomerName, Product, Price, Quantity, TransactionDate]
Index: []


Remembering that what could be formatted was already done, as in 'CustomerName' column. 

### 10 - Rows with Multiple Issues 

With a just a shot is possible to check many rows with multiple issues, like missing values, negative prices, invalid dates, so the focus should be directed to them. 

In [114]:
issues = quick_df.isnull().sum(axis=1) + (quick_df["Price"] < 0) + (~quick_df["TransactionDate"].notnull())
flagged_rows = quick_df[issues > 1]
print("Rows that need attention:\n", flagged_rows)

Rows that need attention:
    TransactionID CustomerName Product  Price  Quantity TransactionDate
4            104          NaN  Tablet   -300       1.0             NaN


This cell doesn't have values for CustomerName and TransactionDate, also the price pointed out on it is negative, so there is no fix at this stage. 

Once the quick check was done, the cleaning will be complete in the next topic. 

### 11 - Addressing Issues Properly

As the code snippets above are only meant for a quick check, this last topics aims to address some issues properly. 

In [123]:
# the function below will sort out the multiple format dates issues
from datetime import datetime

def parse_date(date_str):
    # Handle missing or invalid values
    if date_str in ["nan", "None", ""]:
        return pd.NaT
    
    # First check for the specific format "DD-MM-YYYY"
    if len(date_str) == 10 and date_str[2] == "-" and date_str[5] == "-":
        try:
            return datetime.strptime(date_str, "%d-%m-%Y")
        except ValueError:
            return pd.NaT

    # Attempt parsing with known formats
    formats = ["%Y-%m-%d", "%Y/%m/%d"]
    for fmt in formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    # Return NaT if no format matches
    return pd.NaT

# Ensure TransactionDate is a string
quick_df["TransactionDate"] = quick_df["TransactionDate"].astype(str).str.strip()

# Apply the custom parsing function
quick_df["TransactionDate"] = quick_df["TransactionDate"].apply(parse_date)

# Verify the result
print(quick_df["TransactionDate"])


1   2024-12-01
2   2024-12-01
3   2024-12-01
4          NaT
5   2024-12-01
Name: TransactionDate, dtype: datetime64[ns]


The only 'NaT' value is because there isn't really any value in that cell. 

In [130]:
print(quick_df)

   TransactionID CustomerName Product  Price  Quantity TransactionDate
1            101    Jane Rust  Laptop   1200       1.0      2024-12-01
2            102   June Young   Phone    800       2.0      2024-12-01
3            103    Jane Rust  Laptop   1200       NaN      2024-12-01
4            104          NaN  Tablet   -300       1.0             NaT
5            105   June Young   Phone    850       1.0      2024-12-01
